In [1]:
import pandas as pd
from transformer.dataset import SummaryDataset
from transformer.tokenizer import get_or_build_tokenizer

In [2]:
test_df = pd.read_csv('/home/karan/kj_workspace/kj_ai/AI-learning/attention_is_all_you_need/data/cnn_dailymail/test.csv')

In [3]:
val_input = test_df.article[200:210]

In [4]:
val_summary = test_df.highlights[200:210]

In [ ]:
val_experiment_data_df = pd.DataFrame({'input':val_input, 'summary':val_summary})

In [5]:
val_experiment_data_df.to_csv('experiment_validation_data.csv', index=False)

In [3]:
# test_df.head()

In [4]:
# input = test_df.article[:100]

In [5]:
# summary = test_df.highlights[:100]

In [6]:
# experiment_data_df = pd.DataFrame({'input':input, 'summary':summary})

In [7]:
# experiment_data_df.head()

In [8]:
# experiment_data_df.to_csv('experiment_summary_data.csv', index=False)

In [9]:
experiment_data_df = pd.read_csv('experiment_summary_data.csv')

In [10]:
len(experiment_data_df)

100

In [11]:
for input, summary in zip(experiment_data_df['input'], experiment_data_df['summary']):
    print(input+ ' ' + summary)
    break

Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space 

In [12]:
tokenizer = get_or_build_tokenizer('./tokenizer_exp.json', experiment_data_df)

In [13]:
summary_dataset = SummaryDataset(experiment_data_df, 2000, tokenizer )

In [14]:
from torch.utils.data import DataLoader

In [15]:
train_dataloader = DataLoader(summary_dataset, batch_size=2, shuffle=True)

In [16]:
from transformer.encoder_decoder_model import EncoderDecoder

In [17]:
import torch


In [18]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"


In [19]:
device = 'cpu'

In [20]:
model = EncoderDecoder(device=device, vocab_size=tokenizer.get_vocab_size())

In [21]:
from torch import nn

In [22]:
# Initialize the parameters
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [23]:
model = model.to(device)

In [24]:
device

'cpu'

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=10**-4, eps=1e-9)

In [26]:
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

In [27]:
epochs = 5

In [28]:
from tqdm import tqdm

In [29]:
d = None

In [39]:

for epoch in range(epochs):
    # torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader, desc=f'Processing Epoch {epoch:02d}')
    for batch in batch_iterator:

        encoder_input = batch['encoder_input'].to(device)
        decoder_input = batch['decoder_input'].to(device)
        encoder_mask = batch['encoder_mask'].to(device)
        decoder_mask = batch['decoder_mask'].to(device)

        decoder_output = model(encoder_input, decoder_input, encoder_mask, decoder_mask)

        # decoder_output shape (Batch, seq_len, vocab_size)
        label = batch['label'].to(device) # (Batch, seq_len)

        loss = loss_fn(decoder_output.view(-1, tokenizer.get_vocab_size()), label.view(-1))
        batch_iterator.set_postfix({'loss': f'{loss.item():6.3f}'})

        loss.backward()

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        

Processing Epoch 04: 100%|██████████| 50/50 [04:36<00:00,  5.53s/it, loss=6.306]


In [35]:
tokenizer.get_vocab_size()

5529

In [36]:
d_t = d.view(-1, tokenizer.get_vocab_size())

In [37]:
d_t.shape

torch.Size([4000, 5529])

In [38]:
d.shape

torch.Size([2, 2000, 5529])